In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import jpype
import tempfile
import os
from LocalRuleClassifier import LocalRuleClassifier

In [2]:
# internal config
jar_path = "/LORD-master/lord.jar"
jpype.startJVM(
    jpype.getDefaultJVMPath(),
    "-ea",
    "--enable-native-access=ALL-UNNAMED",
    classpath=[jar_path]
)

In [3]:
datasets = [
    ("mushroom", 1),
    ("bank-marketing", 1),
    ("adult", 1),
    ("page-blocks", 1)
]

model = LocalRuleClassifier(
    metric="MESTIMATE",
    metric_arg=0.1
)

In [4]:
results = []

for name, version in datasets:
    X, y = fetch_openml(name, version=version, as_frame=True, return_X_y=True)
    
    X_raw = X.astype(str)
    y_raw = y.astype(str)    

    
    X_raw_train, X_raw_test, y_train, y_test = train_test_split(
        X_raw, y_raw, test_size=0.1, stratify=y_raw, random_state=42
    )
    
    X_tr = X_raw_train.values
    X_te = X_raw_test.values

    start = time.time()
    model.fit(X_tr, y_train)
    dt = time.time() - start

    y_pred = model.predict(X_te)
    acc = accuracy_score(y_test, y_pred)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    results.append((name, f"{acc:.3f}", f"{dt:.2f}", len(y_test)))

	SearchRuleThread 2 founds 32 rules, finished in 350 ms
	SearchRuleThread 3 founds 29 rules, finished in 350 ms
	SearchRuleThread 0 founds 31 rules, finished in 350 ms
	SearchRuleThread 1 founds 32 rules, finished in 350 ms
	FilterRuleThread 1 finished in 28 ms
	FilterRuleThread 3 finished in 28 ms
	FilterRuleThread 0 finished in 28 ms
	FilterRuleThread 2 finished in 28 ms
	SearchRuleThread 2 founds 5023 rules, finished in 1370 ms
	SearchRuleThread 3 founds 4931 rules, finished in 1370 ms
	SearchRuleThread 1 founds 4957 rules, finished in 1370 ms
	SearchRuleThread 0 founds 4956 rules, finished in 1370 ms
	FilterRuleThread 3 finished in 240 ms
	FilterRuleThread 0 finished in 240 ms
	FilterRuleThread 1 finished in 240 ms
	FilterRuleThread 2 finished in 240 ms
	SearchRuleThread 0 founds 3540 rules, finished in 2966 ms
	SearchRuleThread 2 founds 3602 rules, finished in 2966 ms
	SearchRuleThread 3 founds 3541 rules, finished in 2966 ms
	SearchRuleThread 1 founds 3660 rules, finished in 2966

In [5]:
df_results = pd.DataFrame(results, columns=["Dataset", "Accuracy", "Time (s)","Test Samples"])
df_results

,Dataset,Accuracy,Time (s),Test Samples
0,mushroom,1.000,0.54,813
1,bank-marketing,0.888,1.89,4522
2,adult,0.839,3.51,4885
3,page-blocks,0.922,0.33,548
